Group 19 Final

Introduction:

Data Acquisition:

In [ ]:
import pandas as pd
import nltk
import requests
import twint
import nest_asyncio
import json
import re
from textblob import TextBlob
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
nest_asyncio.apply()

In [ ]:
c = twint.Config()
c.Search = 'Nuclear Energy'
c.Limit = 5000
c.Store_json = True
#c.Output = 'twit_data.json'
twint.run.Search(c)

In [ ]:
c = twint.Config()
c.Search = 'Nuclear Power'
c.Limit = 5000
c.Store_json = True
#c.Output = 'twit_data.json'
twint.run.Search(c)

Data Cleaning:

In [ ]:
f = open(r'twit_data.json', 'rb')
data = [json.loads(line) for line in f]
tweet_df = pd.DataFrame(data)
tweet_df = tweet_df[['tweet','link']]
tweet_df = tweet_df.drop_duplicates()
dataremoved = data[~data.iloc[:,0].str.contains('Ukraine')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('ukraine')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('Ukrainian')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('ukrainian')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('Ukrainians')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('ukrainians')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('Zaporizhzhia')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('war')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('russia')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('Russia')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('russian')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('Russian')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('russians')]
dataremoved = dataremoved[~data.iloc[:,0].str.contains('Russians')]

In [ ]:
cleaned_tweets = []
words = []
for tweet in dataremoved['tweet']:
    clean = re.sub(r"(http[s]?\://\S+)|([\[\(].*[\)\]])|([#@]\S+)|\n", "", tweet)
    clean = re.sub(r"\d", '', clean)
    clean = re.sub(r"'\S+", '', clean)
    clean = clean.replace('.', '').replace(';', '').lower()
    words += re.findall(r"(?:\w+|'|’)+", clean)
    cleaned_tweets.append(clean)

stopwords = nltk.corpus.stopwords.words("english")
standardized = [w for w in words if w not in stopwords]

def wordnet_pos(tag):
    """Map a Brown POS tag to a WordNet POS tag."""

    table = {"N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV, "J": wordnet.ADJ}

    # Default to a noun.
    # if the key doesn't exist, it will return a wordnet.NOUN
    return table.get(tag[0], wordnet.NOUN)

# all pos_tags for lower words
twit_tags = nltk.pos_tag(standardized)

# obtain tags
tags = [wordnet_pos(x[1]) for x in twit_tags]

# create a new sentence
new_text = " ".join(w for w in standardized)
blob = TextBlob(new_text)
# obtain tags
tags = [wordnet_pos(x[1]) for x in blob.pos_tags]

# finalize the lemmatization
new_text = " ".join(x.lemmatize(t) for x, t in zip(blob.words, tags))
# words after lemmatization
standardized_words = TextBlob(new_text)

First Model: Preliminary LDA

Second Model: K-NN

Third Model: VADER

Closing Statements: